In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

class Perceptron:
    def __init__(self, learning_rate=0.01, iterations=20):
        self.lr = learning_rate
        self.iterations = iterations
        self.weights = None
        self.bias = None

    def step_function(self, x):
        return np.where(x >= 0, 1, 0)

    def fit(self, X, y):
        np.random.seed(42)  # For reproducibility
        n_samples, n_features = X.shape

        # Initialize weights randomly
        self.weights = np.random.randn(n_features)
        self.bias = np.random.randn()

        for _ in range(self.iterations):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self.step_function(linear_output)

                # Update rule
                update = self.lr * (y[idx] - y_predicted)
                self.weights += update * x_i
                self.bias += update

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        return self.step_function(linear_output)


def plot_decision_boundary(X, y, model):
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, 0.1),
                           np.arange(x2_min, x2_max, 0.1))

    grid = np.c_[xx1.ravel(), xx2.ravel()]
    predictions = model.predict(grid).reshape(xx1.shape)

    plt.contourf(xx1, xx2, predictions, alpha=0.3, cmap=plt.cm.coolwarm)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm, edgecolor='k')
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.title('Decision Boundary')
    plt.show()


if __name__ == "__main__":
    # Load dataset
    data = pd.read_csv('A5_P1.csv')
    X = data[['X1', 'X2']].values
    y = data['y'].values

    # Initialize and train perceptron
    perceptron = Perceptron(learning_rate=0.01, iterations=20)
    perceptron.fit(X, y)

    # Plot decision boundary
    plot_decision_boundary(X, y, perceptron)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Activation functions and their derivatives

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1 - np.tanh(x) ** 2

def softmax(x):
    exps = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exps / np.sum(exps, axis=1, keepdims=True)

# Loss function: Mean Squared Error

def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def mse_derivative(y_true, y_pred):
    return (y_pred - y_true)

# Load dataset
iris = load_iris()
X = iris.data
y = iris.target

# One-hot encode the labels
y_onehot = np.zeros((y.size, y.max() + 1))
y_onehot[np.arange(y.size), y] = 1

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Network architecture
input_size = X_train.shape[1]
hidden_size = 5
output_size = 3

# Initialize weights and biases
np.random.seed(42)
W1 = np.random.randn(input_size, hidden_size)
b1 = np.random.randn(hidden_size)
W2 = np.random.randn(hidden_size, output_size)
b2 = np.random.randn(output_size)

# Training parameters
learning_rate = 0.01
epochs = 1000

# Lists to store loss and accuracy values
train_losses, test_losses = [], []
train_accuracies, test_accuracies = [], []

# Training loop
for epoch in range(epochs):
    # Forward propagation (Training)
    Z1 = np.dot(X_train, W1) + b1
    A1 = tanh(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)

    # Calculate loss and accuracy for training
    train_loss = mse(y_train, A2)
    train_losses.append(train_loss)
    train_accuracy = np.mean(np.argmax(A2, axis=1) == np.argmax(y_train, axis=1))
    train_accuracies.append(train_accuracy)

    # Backward propagation
    dZ2 = mse_derivative(y_train, A2)
    dW2 = np.dot(A1.T, dZ2)
    db2 = np.sum(dZ2, axis=0)

    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * tanh_derivative(Z1)
    dW1 = np.dot(X_train.T, dZ1)
    db1 = np.sum(dZ1, axis=0)

    # Update weights and biases
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

    # Testing
    Z1_test = np.dot(X_test, W1) + b1
    A1_test = tanh(Z1_test)
    Z2_test = np.dot(A1_test, W2) + b2
    A2_test = softmax(Z2_test)

    # Calculate loss and accuracy for testing
    test_loss = mse(y_test, A2_test)
    test_losses.append(test_loss)
    test_accuracy = np.mean(np.argmax(A2_test, axis=1) == np.argmax(y_test, axis=1))
    test_accuracies.append(test_accuracy)

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Train Acc: {train_accuracy:.4f}, Test Acc: {test_accuracy:.4f}')

# Plotting Loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(test_losses, label='Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss vs. Epochs')

# Plotting Accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(test_accuracies, label='Testing Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy vs. Epochs')

plt.show()

